In [1]:
import os
from pathlib import Path
os.chdir(Path.cwd().parent)

In [2]:
import pandas as pd
import numpy as np
# from datetime import datetime
from data_loader import scores_df
from fit import fit_statistical_model

null performances after coercion: 0
after saturation filter 1025
after filter num benchmarks 759
after merge with model versions 759
after merge with benchmark dates 759
Original number of rows: 759
Number of rows after aggregation: 682


In [3]:
anchor_benchmark = "Winogrande"
anchor_difficulty = 0
anchor_slope = 1
df1, df_cm1, df_db1 = fit_statistical_model(scores_df, anchor_benchmark, anchor_difficulty, anchor_slope)

# Convert date strings to datetime objects
df_cm1['date_obj'] = pd.to_datetime(df_cm1['date'])

`ftol` termination condition is satisfied.
Function evaluations 46, initial cost 2.4296e+01, final cost 1.6472e+00, first-order optimality 1.63e-05.


In [30]:
a = 52.94
b = 48

df_cm1[df_cm1['model'].isin(['text-davinci-001', 'gpt-4-0613'])]['estimated_capability']*b + a

15    120.107576
94    100.104300
Name: estimated_capability, dtype: float64

In [35]:
def get_rank(df, date_col='date', score_col='eci'):
    """
    For each row, compute the rank of 'score' among all rows 
    with the same or earlier 'date'.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing at least two columns: date and score.
    date_col : str, default 'date'
        Column name for the date.
    score_col : str, default 'eci'
        Column name for the score.

    Returns
    -------
    pd.Series
        A pandas Series containing the rank for each row.
    """
    # Ensure dates are sorted
    df_sorted = df.sort_values(by=date_col).reset_index(drop=True)
    
    ranks = []
    for i, row in df_sorted.iterrows():
        # Subset to all rows with same or earlier date
        subset = df_sorted.loc[df_sorted[date_col] <= row[date_col], score_col]
        # Rank current score within that subset
        rank = subset.rank(ascending=False, method='min').iloc[i]
        ranks.append(rank)
    
    return pd.Series(ranks, index=df_sorted.index)

In [37]:
df_cm1['eci'] = np.round(df_cm1['estimated_capability']*b + a, 1)
df_cm1['eci_no_decimals'] = np.round(df_cm1['eci'])
df_cm1['rank'] = get_rank(df_cm1, 'date', 'eci')
df_cm1['is_frontier'] = df_cm1['rank'] == 1
df_cm1['top_5'] = df_cm1['rank'] <= 5

df_cm1.to_csv("output/test.csv")

In [ ]:
# Check the top 5 at each date, and count the number of time we get ties

# Get unique dates and sort them
unique_dates = sorted(df_cm1['date_obj'].unique())

tie_counts = []
date_tie_info = []

for date in unique_dates:
    # Get all models released on this date or earlier
    date_data = df_cm1[df_cm1['date_obj'] <= date].copy()
    
    # Sort by ECI (descending) and get top 5
    date_data_sorted = date_data.sort_values('eci_no_decimals', ascending=False)
    top_5_data = date_data_sorted.head(5).copy()
    
    if len(top_5_data) > 0:
        # Count unique ECI values in top 5
        unique_eci_values = top_5_data['eci_no_decimals'].nunique()
        total_top_5 = len(top_5_data)
        
        # If we have fewer unique values than total models, we have ties
        ties_at_date = total_top_5 - unique_eci_values
        
        tie_counts.append(ties_at_date)
        date_tie_info.append({
            'date': date,
            'total_top_5': total_top_5,
            'unique_eci_values': unique_eci_values,
            'ties': ties_at_date,
            'eci_values': sorted(top_5_data['eci_no_decimals'].unique(), reverse=True)
        })
        
        if ties_at_date > 0:
            print(f"Date {date.strftime('%Y-%m-%d')}: {ties_at_date} ties in top 5")
            print(f"  Top 5 ECI values: {sorted(top_5_data['eci_no_decimals'].unique(), reverse=True)}")
            print(f"  Models: {sorted(top_5_data['model'].tolist())}")
            print()

print(f"\nSummary:")
print(f"Total dates with top 5 models: {len(tie_counts)}")
print(f"Total ties across all dates: {sum(tie_counts)}")
print(f"Average ties per date: {np.mean(tie_counts):.2f}")
print(f"Max ties at any date: {max(tie_counts) if tie_counts else 0}")

# Create a summary DataFrame
tie_summary = pd.DataFrame(date_tie_info)
print(f"\nDetailed tie analysis:")
print(tie_summary[['date', 'total_top_5', 'unique_eci_values', 'ties']].head(10))




Summary:
Total dates with top 5 models: 34
Total ties across all dates: 0
Average ties per date: 0.00
Max ties at any date: 0

Detailed tie analysis:
        date  total_top_5  unique_eci_values  ties
0 2023-02-24            1                  1     0
1 2023-02-27            1                  1     0
2 2023-03-14            1                  1     0
3 2023-03-15            1                  1     0
4 2023-07-18            2                  2     0
5 2023-09-27            1                  1     0
6 2024-02-21            2                  2     0
7 2024-02-26            1                  1     0
8 2024-03-07            1                  1     0
9 2024-04-09            1                  1     0
